In [ ]:
import os
import sys

In [ ]:
home = os.environ['HOME']
work_path = os.path.join(home, 'far/farMe/uFunc')
src_path = os.path.join(home, 'far/PHARE')
build_path = os.path.join(home, 'far/builds/release/ufunc')

In [ ]:
sys.path.append(os.path.join(src_path, "pyphare"))

In [ ]:
import subprocess
import pyphare
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
from pyphare.core.operators import dot, cross, sqrt, modulus, grad
from pyphare.core.ufuncs import gFilt, gF, peakIds
import numpy as np

In [ ]:
run_path = os.path.join(work_path, 'wp')

In [ ]:
if os.path.isdir(run_path):
    files = os.listdir(run_path)
else:
    files = []

In [ ]:
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += os.pathsep + os.path.join(src_path, "pyphare")
else:
    os.environ['PYTHONPATH'] = os.pathsep + os.path.join(src_path, "pyphare")
os.environ['PYTHONPATH'] += os.pathsep + build_path

In [ ]:
if 'ions_charge_density.h5' not in files :
    os.chdir(work_path)
    subprocess.run(['mpirun', '-n', '4', '/usr/bin/python3', work_path+'/wp.py', ], env=os.environ)    

In [ ]:
run  = Run(run_path)
time = 0.0

In [ ]:
N = run.GetNi(time)
type(N)

In [ ]:
for pls in N.patch_levels:
    for il, pl in pls.items():
        for p in pl:
            print(type(p))
            print(p, np.sqrt(p))
            # for k, v in p.patch_datas.items():
            #     print(np.sqrt(v))

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

N.plot(qty='value', ax=ax, ls='solid', lw=1.0, color='tab:blue', xlabel='X ($d_i$)', ylabel='$V_X$ ($v_A$)')

# define the domain, grid and function to work on

In [ ]:
step = 0.2         # grid size
Nx = 50            # total number of grid point in the full domain
L = Nx*step        # total size of the domain
num_of_ghosts = 2  # num of ghosts
# half = 20          # position in index of where the first box is ending and the second is starting
# sep = half*step    # where is the separation between the 2 patches in physical units

In [ ]:
def f(x):
    y = x+0.4
    return np.exp(-0.1*y**2)*y

def g(x):
    return f(x)+np.random.rand(x.shape[0])*0.4-0.2

def h(x):
    return (1.4+np.sin(2*np.pi*x/L))*0.4

In [ ]:
x = np.arange(0, L+2*num_of_ghosts*step, step)-num_of_ghosts*step+0.5*step
y = f(x)
z = g(x)
u = h(x)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 3))

plt.plot(x, y)
plt.plot(x, u)
# plt.plot([sep, sep], [0, 1.3], 'k:')

# define the mock of a patch and a hierarchy

In [ ]:
class Patch():
    def __init__(self, data, idx, name="unnamed", num_of_ghosts=2):
        assert(data.shape == idx.shape)
        self.data = np.asarray(data)
        self.nx = self.data.shape[0]
        self.idx = np.asarray(idx)
        self.name = name
        self.num_of_ghosts = num_of_ghosts

    def __add__(self, other):
        if _disables_array_ufunc(other):
            return NotImplemented
        assert(self.data.shape == other.data.shape)
        return Patch(np.add(self.data, other.data), self.idx, self.num_of_ghosts)

    def __mul__(self, other):
        if _disables_array_ufunc(other):
            return NotImplemented
        return np.multiply(self, other)

    def __array__(self, dtype=None):
        """ this is the numpy array protocol"""
        return np.asarray(self.data, dtype=dtype)

    def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
        print(f"__array_function__ de Patch appelé pour {ufunc.__name__}")
        if method != "__call__":
            raise NotImplementedError

        unwrapped = [i.data if isinstance(i, Patch) else i for i in inputs]
        result = getattr(ufunc, method)(*unwrapped, **kwargs)

        if isinstance(result, np.ndarray):
            return Patch(result, self.idx, name=ufunc.__name__+"@"+self.name, num_of_ghosts=self.num_of_ghosts)

    def __array_function__(self, func, types, args, kwargs):
        print(f"__array_function__ de Patch {func.__name__} appelé avec {[getattr(a, 'name', a) for a in args]}")
    
        unwrapped = [a.data if isinstance(a, Patch) else a for a in args]
        result = func(*unwrapped, **kwargs)
    
        if isinstance(result, np.ndarray):
            return Patch(result, self.idx, name=func.__name__+"@"+self.name, num_of_ghosts=self.num_of_ghosts)
        else:
            return result

    def __repr__(self):
        return """
        name : {0}
        data : {1}
        idx  : {2}
        nx   = {3}""".format(self.name, self.data, self.idx, self.nx)

In [ ]:
sep = [20, 30, ]                      # contain the index of the boundaries of each patches w. a total number of patches == len(sep)+1
bounds = sep                      # these are all the separators
bounds = [0]+bounds               # then add the first point
bounds.append(Nx)                 # and the last one
boxes = [i*step for i in bounds]  # same as bounds bu in physical units, not indices
print(bounds, boxes)

In [ ]:
def fillPatch(id, func, box, step, num_of_ghosts=num_of_ghosts, name="unnamed"):
    idx = np.arange((box[0]-num_of_ghosts+0.5)*step, (box[1]+num_of_ghosts+0.5)*step, step)
    data = func(idx)
    return Patch(data, idx, name)

In [ ]:
num_of_patches = len(bounds)-1
patches = []
print(num_of_patches)

for i in range(num_of_patches):
    print(i, bounds[i:i+2])
    patches.append(fillPatch(i, f, bounds[i:i+2], step, name=(f.__name__+str(i))))

In [ ]:
patches

In [ ]:
# def _list_mean(x, *args, **kwargs):
#     if not isinstance(x, Hierarchy):
#         return NotImplemented
# 
#     return NamedArrayList([
#         np.mean(a, *args, **kwargs)
#         for a in x.arrays
#     ])
# 
# def _list_add(x, y):
#     if not isinstance(x, Hierarchy) or not isinstance(y, Hierarchy):
#         return NotImplemented
#     if len(x) != len(y):
#         raise ValueError("Hierarchy length mismatch")
# 
#     return Hierarchy([
#         np.add(a, b) for a, b in zip(x.arrays, y.arrays)
#     ])
# 
# _LIST_HANDLED_FUNCTIONS = {
#     np.mean: _list_mean,
#     np.add:  _list_add,   # optional, ufunc already covers this
# }

In [ ]:
class Hierarchy():
    def __init__(self, patches, boxes):
        if not all(isinstance(a, Patch) for a in patches):
            raise TypeError("each elements of the patches list has to be a Patch")
        self.patches = list(patches)
        # self.numOfPatches = len(self.patches)
        self.boxes = boxes

    def __len__(self):
        return len(self.patches)

    def __repr__(self):
        r = ""
        for i, p in enumerate(self.patches):
            o = "{0} \n".format(p)
            r += o
        return r

    def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
        print(f"__array_function__ de Hierarchy appelé pour {ufunc.__name__}")
        if method != "__call__":
            return NotImplemented

        # Collect per-argument sequences
        seqs = []
        for x in inputs:
            if isinstance(x, Hierarchy):
                seqs.append(x.patches)
            else:
                seqs.append([x] * len(self))

        # Outer length must match
        n = len(seqs[0])                        # length of the first hierarchy
        if not all(len(s) == n for s in seqs):  # each of the other hierarchies have to be homogeneous
            raise ValueError("Hierarchy length mismatch")

        # Pairwise application
        result = [
            ufunc(*elems, **kwargs)
            for elems in zip(*seqs)
        ]

        return Hierarchy(result, self.boxes)

    def __array_function__(self, func, types, args, kwargs):
        print(f"__array_function__ de Hierarchy {func.__name__} appelé avec {[getattr(a, 'name', a) for a in args]}")
        if func not in _LIST_HANDLED_FUNCTIONS:
            return NotImplemented
        # return _LIST_HANDLED_FUNCTIONS[func](*args, **kwargs)
        return func(*args, **kwargs)

    def __repr__(self):
        s = ""
        for i, p in enumerate(self.patches):
            s += p.__repr__()+"\n"
        return s

    def plot(self, **kwargs):
        fig, ax = plt.subplots(figsize=(12, 3))

        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
        markers = ["o", ".", "s", "*", "p", "P", "X", "D", "h", "p"]
        
        for i, p in enumerate(self.patches):
            ax.plot(p.idx, p.data, color=colors[i], marker=markers[i])

        ax.axvline(boxes[0] , ls='-', c='k')
        for x in boxes[1:-1]:
            ax.axvline(x, ls=':', c='k')
        ax.axvline(boxes[-1], ls='-', c='k')

In [ ]:
hy = Hierarchy(patches, boxes)

In [ ]:
hy.plot()

In [ ]:
new_patches = [np.sqrt(p) for p in patches]

In [ ]:
hn = Hierarchy(new_patches, boxes)

In [ ]:
hn.plot()

In [ ]:
hm = np.sqrt(hy)

In [ ]:
hm.plot()